In [21]:
"""
import pandas as pd
import yfinance as yf
from finvizfinance.screener.overview import Overview

def get_combined_top_stocks(countries=['USA', 'Canada'], limit_per_country=100):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching top rated stocks for: {country}...")
        
        try:
            foverview = Overview()
            
            # Use the correct filter syntax we found earlier
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- FIX STARTS HERE ---
    # 1. Inspect available columns to avoid guessing
    available_columns = combined_df.columns.tolist()
    print(f"\nDebug: Columns found in data: {available_columns}")

    # 2. Determine which 'Recommendation' column name is being used (if any)
    recom_col = None
    if 'Recom' in available_columns:
        recom_col = 'Recom'
    elif 'Analyst Recom' in available_columns:
        recom_col = 'Analyst Recom'
        
    # 3. Sort if we found the column
    if recom_col:
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # 4. safely build the list of columns to return
    # We only include columns that actually exist in 'available_columns'
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country']
    
    # Add the recommendation column to our list if it exists
    if recom_col:
        desired_cols.append(recom_col)

    # Filter columns safely
    final_cols = [c for c in desired_cols if c in available_columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_north_america = get_combined_top_stocks(['USA', 'Canada'])

if not top_north_america.empty:
    print(f"\nFound {len(top_north_america)} stocks.")
    print(top_north_america)
    
    # Get tickers
    ticker_list = top_north_america['Ticker'].tolist()
    print(f"\nTicker List: {ticker_list}")
else:
    print("DataFrame is empty.")
"""

    

'\nimport pandas as pd\nimport yfinance as yf\nfrom finvizfinance.screener.overview import Overview\n\ndef get_combined_top_stocks(countries=[\'USA\', \'Canada\'], limit_per_country=100):\n    all_stocks = []\n\n    for country in countries:\n        print(f"Fetching top rated stocks for: {country}...")\n\n        try:\n            foverview = Overview()\n\n            # Use the correct filter syntax we found earlier\n            filters_dict = {\n                \'Analyst Recom.\': \'Strong Buy (1)\',\n                \'Country\': country\n            }\n            foverview.set_filter(filters_dict=filters_dict)\n\n            # Get results\n            df_results = foverview.screener_view()\n\n            if not df_results.empty:\n                df_results[\'Source_Country\'] = country\n                all_stocks.append(df_results)\n            else:\n                print(f"   No stocks found for {country}.")\n\n        except Exception as e:\n            print(f"   Error fetching

In [22]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['Canada'], limit_per_country=200):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The Rating Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 2.0 (Buy or Strong Buy)
        print(f"Filtering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] <= 2.0]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks_canada = get_filtered_picks(['Canada'])

if not top_picks_canada.empty:
    # Ensure Price is numeric (so you can sort it yourself in the Data Viewer)
    top_picks_canada['Price'] = pd.to_numeric(top_picks_canada['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks_canada)} 'Buy' rated Canadian stocks.")
    print("Variable 'top_picks' is ready. Click it in the Jupyter Variables view.")

else:
    print("DataFrame is empty.")

Fetching candidates for: Canada...
[Info] loading page [###########################---] 8/9 
Success! Found 162 'Buy' rated Canadian stocks.
Variable 'top_picks' is ready. Click it in the Jupyter Variables view.


c:\Users\jdcc3\AppData\Local\Programs\Python\Python313\Lib\site-packages\finvizfinance\screener\base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


In [23]:
"""
import pandas as pd
from finvizfinance.screener.overview import Overview

# Top picks for Canada with refined filtering
def get_filtered_picks(countries=['Canada'], limit_per_country=200):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            # We ask for 'Buy or better' to get a mix of Strong Buys (1.0) and regular Buys (2.0)
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    
    # A. Identify the correct column name
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # B. Convert column to numbers (force any text to become NaN)
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # C. APPLY THE STRICT FILTER
        # Logic: 1.0 = Strong Buy, 2.0 = Buy, 3.0 = Hold.
        # This keeps 2.0 (Buy) and better.
        print(f"\nFiltering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] <= 2.0]
        
        # D. Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['Canada'])

if not top_picks.empty:
    # Ensure Price is numeric so you can sort by Price in the Data Viewer
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nFound {len(top_picks)} 'Buy' rated stocks.")
    print("Variable 'top_picks' is ready. Click it in the Jupyter Variables view.")
    
    # Get tickers (Optional)
    ticker_list = top_picks['Ticker'].tolist()

else:
    print("DataFrame is empty.")
    """

'\nimport pandas as pd\nfrom finvizfinance.screener.overview import Overview\n\n# Top picks for Canada with refined filtering\ndef get_filtered_picks(countries=[\'Canada\'], limit_per_country=200):\n    all_stocks = []\n\n    for country in countries:\n        print(f"Fetching candidates for: {country}...")\n\n        try:\n            foverview = Overview()\n\n            # --- STEP 1: Broaden the initial search ---\n            # We ask for \'Buy or better\' to get a mix of Strong Buys (1.0) and regular Buys (2.0)\n            filters_dict = {\n                \'Analyst Recom.\': \'Buy or better\',\n                \'Country\': country\n            }\n            foverview.set_filter(filters_dict=filters_dict)\n\n            # Get results\n            df_results = foverview.screener_view()\n\n            if not df_results.empty:\n                df_results[\'Source_Country\'] = country\n                all_stocks.append(df_results)\n            else:\n                print(f"   No st

In [24]:
"""import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            # We ask for 'Buy or better' to get a mix of Strong Buys (1.0) and regular Buys (2.0)
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    
    # A. Identify the correct column name (usually 'Recom' or 'Analyst Recom')
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # B. Convert column to numbers (force any text to become NaN)
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # C. APPLY THE STRICT FILTER
        # Logic: 1.0 = Strong Buy, 2.0 = Buy, 3.0 = Hold.
        # We keep anything <= 2.5 to strictly exclude 'Hold' (3.0) and worse.
        print(f"\nFiltering out 'Hold' or worse (Rating > 2.0)...")
        combined_df = combined_df[combined_df[recom_col] < 1.5]
        
        # D. Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns for display
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    print(f"\nFound {len(top_picks)} 'Buy' rated stocks.")
    
    # --- STEP 3: The View Fix ---
    # This forces VS Code/Terminal to show ALL rows
    #pd.set_option('display.max_rows', None)
    
    print(top_picks)
    
    # Get tickers
    ticker_list = top_picks['Ticker'].tolist()
    #print(f"\nTicker List: {ticker_list}") # Optional: Uncomment to see list
else:
    print("DataFrame is empty.")
    """

'import pandas as pd\nfrom finvizfinance.screener.overview import Overview\n\ndef get_filtered_picks(countries=[\'USA\'], limit_per_country=5000):\n    all_stocks = []\n\n    for country in countries:\n        print(f"Fetching candidates for: {country}...")\n\n        try:\n            foverview = Overview()\n\n            # --- STEP 1: Broaden the initial search ---\n            # We ask for \'Buy or better\' to get a mix of Strong Buys (1.0) and regular Buys (2.0)\n            filters_dict = {\n                \'Analyst Recom.\': \'Buy or better\',\n                \'Country\': country\n            }\n            foverview.set_filter(filters_dict=filters_dict)\n\n            # Get results\n            df_results = foverview.screener_view()\n\n            if not df_results.empty:\n                df_results[\'Source_Country\'] = country\n                all_stocks.append(df_results)\n            else:\n                print(f"   No stocks found for {country}.")\n\n        except Excep

In [25]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Buy or better',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 1.5 (Strong Buys)
        print(f"Filtering out 'Hold' or worse (Rating > 1.5)...")
        combined_df = combined_df[combined_df[recom_col] < 1.5]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # Ensure Price is numeric (Crucial for sorting in Data Viewer)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks)} candidates.")
    print("Variable 'top_picks' is ready for the Data Viewer.")

else:
    print("DataFrame is empty.")

Fetching candidates for: USA...
[Info] loading page [##############################] 149/150 
Success! Found 2996 candidates.
Variable 'top_picks' is ready for the Data Viewer.


In [26]:
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    mid_picks = top_picks[(top_picks['Price'] > 5) & (top_picks['Price'] < 30)]

    print(f"\nFound {len(mid_picks)} stocks between $5 and $30.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 1056 stocks between $5 and $30.


In [27]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    cheap_picks = top_picks[(top_picks['Price'] < 5)]

    print(f"\nFound {len(cheap_picks)} stocks between $5 and $30.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 701 stocks between $5 and $30.


In [28]:
if not top_picks.empty:
    # 1. FIX: Do NOT convert 'Sector' to numeric. 
    # (Only 'Price' needs to be numeric if you filter by price later)
    # top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce') 

    # 2. Filter for Sector = 'Technology'
    # We use .str.contains to be safe (catches 'Technology', 'Technology Services', etc.)
    tech_picks = top_picks[top_picks['Sector'].str.contains('Technology', case=False, na=False)]

    print(f"\nFound {len(tech_picks)} Technology stocks.")
    
    # 3. View in Variable Explorer (VS Code)
    # You can now open 'tech_picks' in the Data Viewer
else:
    print("DataFrame is empty.")


Found 456 Technology stocks.


In [29]:
"""
import yfinance as yf
import time

# Loop through EVERY symbol in the list (removed the [:3])
for symbol in ticker_list:
    
    # 1. Fetch data
    stock = yf.Ticker(symbol)
    print(f"\n--- Checking {symbol} ---")
    
    try:
        # 2. Get the upgrades/downgrades
        upgrades = stock.upgrades_downgrades
        
        if not upgrades.empty:
            # Filter for 2025 actions using the .index fix
            recent = upgrades[upgrades.index >= '2025-11-01']
            
            if not recent.empty:
                print(recent)
            else:
                print(f"No actions for {symbol} in 2025.")
        else:
            print(f"No analyst history found for {symbol}.")
            
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    # 3. CRITICAL: Sleep for 1 second to avoid getting banned
    time.sleep(1)
"""

'\nimport yfinance as yf\nimport time\n\n# Loop through EVERY symbol in the list (removed the [:3])\nfor symbol in ticker_list:\n\n    # 1. Fetch data\n    stock = yf.Ticker(symbol)\n    print(f"\n--- Checking {symbol} ---")\n\n    try:\n        # 2. Get the upgrades/downgrades\n        upgrades = stock.upgrades_downgrades\n\n        if not upgrades.empty:\n            # Filter for 2025 actions using the .index fix\n            recent = upgrades[upgrades.index >= \'2025-11-01\']\n\n            if not recent.empty:\n                print(recent)\n            else:\n                print(f"No actions for {symbol} in 2025.")\n        else:\n            print(f"No analyst history found for {symbol}.")\n\n    except Exception as e:\n        print(f"Error fetching {symbol}: {e}")\n\n    # 3. CRITICAL: Sleep for 1 second to avoid getting banned\n    time.sleep(1)\n'